# Fine-tune SantaCoder on YAML
Sources:
* https://github.com/loubnabnl/santacoder-finetuning
* https://colab.research.google.com/drive/1UMjeXHwOldpLnWjdm1499o2IYy0RgeTw?usp=sharing



### Install required libraries

In [1]:
!pip install transformers datasets torch
!pip install huggingface_hub
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 65.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 269.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 276.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 147.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 290.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 289.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 232.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 253.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available:

### Log in to HuggingFace to access The Stack

In [2]:
import os
from huggingface_hub import login
# Note: Prefer notebook_login() but this was not prompting properly
token=os.environ.get('HUGGINGFACE_TOKEN')
login(token)


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


### Import required libraries

In [3]:
import torch
from dataclasses import dataclass
from datasets import load_dataset
from torch.utils.data import IterableDataset
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed
)
from dataset import ConstantLengthDataset

In [4]:
import torch
import os

print("PyTorch version:", torch.__version__)

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
    print("Number of GPU:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Current Device : {torch.cuda.current_device}")
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
        print(torch.cuda.get_device_properties(i))
        print(f"Memory GB: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        print(f"GPU Allocated: {torch.cuda.memory_allocated(i) / 1024 ** 3:.2f} GB")
        print(f"GPU Cached:    {torch.cuda.memory_reserved(i) / 1024 ** 3:.2f} GB")
        
        
else:
    print("CUDA is not available. No GPU detected.")


print(torch.cuda._get_device_index)
print(torch.cuda.current_device)

#torch.cuda.set_device(3)


CUDA_VISIBLE_DEVICES = os.environ.get('CUDA_VISIBLE_DEVICES')
print(CUDA_VISIBLE_DEVICES)

PyTorch version: 2.0.1+cu118
CUDA is available.
Number of GPU: 1
Current Device : <function current_device at 0x7f5d1005ae50>
Device 0: NVIDIA A10G
_CudaDeviceProperties(name='NVIDIA A10G', major=8, minor=6, total_memory=22502MB, multi_processor_count=80)
Memory GB: 21.98 GB
GPU Allocated: 0.00 GB
GPU Cached:    0.00 GB
<function _get_device_index at 0x7f5d101d4ca0>
<function current_device at 0x7f5d1005ae50>
3


In [5]:
!nvidia-smi

Fri Mar  8 09:36:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1B.0 Off |                    0 |
|  0%   27C    P0             66W /  300W |   21346MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Specify the original pre-trained model and fine-tuned model

In [6]:
pretrained_id = "bigcode/santacoder"
model_id = "santacoder-finetuned-the-stack-yaml"

### Load the tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Load the model

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             trust_remote_code=True, 
                                             use_cache=False)

### Create a minimal Ansible dataset from text file

In [9]:
dataset_dict = load_dataset("text", 
                            data_files="ansible-examples-ec2.txt",
                            sample_by="paragraph")
dataset = dataset_dict['train']

### Calculate characters per token

In [10]:
examples, total_characters, total_tokens = 500, 0, 0

for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example['text'])
    total_tokens += len(tokenizer(example['text']).tokens())

characters_per_token = total_characters / total_tokens
print(characters_per_token)

  0%|          | 0/500 [00:00<?, ?it/s]

2.9357690041249263


### Create a test split

In [11]:
dataset = dataset.train_test_split(test_size=0.1)
train_ds = dataset["train"]
valid_ds = dataset["test"]
len(train_ds), len(valid_ds)

(15, 2)

In [12]:
train_ds = train_ds.shuffle(seed=555)

train_dataset = ConstantLengthDataset(
        tokenizer, train_ds, infinite=True, seq_length=1024, chars_per_token=characters_per_token, content_field="text" 
    )
valid_dataset = ConstantLengthDataset(
        tokenizer, valid_ds, infinite=False, seq_length=1024, chars_per_token=characters_per_token, content_field="text"  
    )

### Define the training arguments

In [13]:
training_args = TrainingArguments(
        output_dir="santacoder-finetuned-the-stack-ansible-ec2",
        dataloader_drop_last=True,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        optim="adafactor",
        evaluation_strategy="steps",
#        max_steps=5000,
        max_steps=1000,
        eval_steps=500,
        save_steps=500,
        logging_steps=10,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        weight_decay=0.05,
        fp16=False,
        push_to_hub=False
)

### Train

In [14]:
train_dataset.start_iteration = 0

In [15]:
trainer = Trainer(
    #tokenizer=tokenizer,
    model=model, args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=valid_dataset
)

In [16]:
trainer.train()

Step,Training Loss,Validation Loss


ValueError: Batch does not contain any data (`None`). At the end of all iterable data available before expected stop iteration.

In [ ]:
trainer.save_model() # this creates the config.json file etc you need to run generation tests